### Extraction des features avec RESNET50

In [21]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from sparkdl import DeepImageFeaturizer

In [6]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '-- packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell'

In [7]:
#spark configuration
conf = SparkConf().set('spark.executor.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
 set('spark.driver.extraJavaOptions','-Dcom.amazonaws.services.s3.enableV4=true'). \
 setAppName('pyspark_aws_project8').setMaster('local[*]')

In [16]:
#sc=SparkContext(conf=conf)
#sc.setSystemProperty('com.amazonaws.services.s3.enableV4', 'true')

In [13]:
accessKeyId='xxxxxxxxxx'
secretAccessKey='xxxxxxxxxxxxxxx'

In [18]:
hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set('fs.s3a.access.key', accessKeyId)
hadoopConf.set('fs.s3a.secret.key', secretAccessKey)
hadoopConf.set('fs.s3a.endpoint', 's3-eu-west-1.amazonaws.com')
hadoopConf.set('fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')

NameError: name 'sc' is not defined

In [ ]:
spark=SparkSession(sc)

In [20]:
datasets_echant_path = 's3a:\\my-project8-image\\donnees_image_echant\\fruits-360_dataset\\fruits-360\\Training\\'

In [ ]:
# Create an empty spark dataframe
emp_RDD = spark.sparkContext.emptyRDD()
 
# Defining the schema of the DataFrame
columns1 = StructType([StructField('image',
                                   StructType([
                                       StructField("origin", StringType(), True),
                                       StructField("height", IntegerType(), True),
                                       StructField("width", IntegerType(), True),
                                       StructField("nChannels", IntegerType(), True),
                                       StructField("mode", IntegerType(), True),
                                       StructField("data", BinaryType(), True),
                                       ]), True,),
                       StructField('label', StringType(), False)])
 
# Creating an empty DataFrame
empty_df = spark.createDataFrame(data=emp_RDD,
                                         schema=columns1)

In [ ]:
# Download all pictures with labels

df = empty_df
for dirname, _, _ in os.walk(datasets_echant_path):
    if dirname != datasets_echant_path:
        df_img = spark.read.format("image").load(dirname).withColumn("label", lit(dirname.split('/')[-1]))
        df = df.union(df_img)

In [ ]:
# Extract features from images with Resnet50
featurizer = DeepImageFeaturizer(inputCol="image",
                                 outputCol="features",
                                 modelName="ResNet50")

df_feat = spark.createDataFrame(featurizer.transform(df.drop('label'))).WithColum('label',df.label)

### Réduction de dimension par PCA

In [ ]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
import numpy as np
from pyspark.ml.feature import StandardScaler
import pyspark.sql.functions as f
import pandas as pd
from pyspark.sql import Row
from pyspark.ml.feature import VectorAssembler

In [ ]:
cols = df_feat.drop('label').columns

In [ ]:
# Create vector columns
assembler = VectorAssembler(inputCols=cols, outputCol = 'features')
output_dat = assembler.transform(df_feat).select('label', 'features')

In [ ]:
# Apply PCA
pca = PCA(k=88, inputCol = "features", outputCol="pcaFeatures")

model = pca.fit(output_dat)
pca_features = model.transform(output_dat).select('label', df_feat.label)

In [8]:
from sklearn.decomposition import PCA
# pca - keep 90% of variance
pca = PCA(0.90)
principal_components = pca.fit_transform(data_all.drop("label"))

In [9]:
principal_df = pd.concat([data_all.label, pd.DataFrame(data = principal_components)], axis=1)
print(principal_df.shape)

(1310, 174)


In [12]:
principal_df.head()

,label,0,1,2,3,4,5,6,7,8,...,163,164,165,166,167,168,169,170,171,172
0,Apple Braeburn,-126.079216,-20.465063,2.109932,-16.983776,20.552832,-55.029251,11.705690,-68.226257,-15.284904,...,-2.001430,5.611289,2.469967,13.452518,4.699523,-6.445440,-2.791751,-1.687784,-9.780011,-4.474288
1,Apple Braeburn,-159.821060,-49.361229,26.784433,1.381939,0.252728,-0.365676,50.401302,-12.385925,-82.419731,...,-0.891619,-5.292984,-4.847217,-1.164312,1.676399,9.762865,-10.650976,-6.978362,-5.724542,-0.335709
2,Apple Braeburn,-163.084946,-50.975998,32.951340,-2.983782,-2.024314,7.104094,55.642479,-7.860163,-88.528183,...,-0.831624,-1.517816,-5.190097,-5.411776,3.861518,8.233482,-6.704696,-2.499938,-2.530610,1.072610
3,Apple Braeburn,-160.648453,-49.477711,42.307613,5.905721,-4.085958,10.723994,53.374607,-8.554770,-100.312035,...,-2.856441,-0.888656,-4.545717,-5.491290,2.400442,7.536231,-6.292813,-6.892319,-2.751713,-4.750124
4,Apple Braeburn,-159.546143,-49.895878,53.068077,10.417496,-6.057590,9.659484,55.258743,-11.540544,-100.696190,...,-1.102456,-0.746603,-2.300404,-6.746674,2.137212,6.150426,-5.815139,-6.398622,0.068932,-3.145939


### Stockage en fichier csv

In [11]:
principal_df.to_csv(path+"sortie_pandas.csv")